In [1]:
from scipy.stats import hypergeom
from bioservices import QuickGO
import matplotlib.pyplot as plt
import markov_clustering as mc
import scipy.special
import networkx as nx
import pandas as pd
import numpy as np
import igraph
import json
import copy

In [6]:
def save_json(name, data):
    with open(name, "w") as f:
         json.dump(data, f)
    print("done.")

In [7]:
def load_json(name, data=None):
    with open(name, "r") as f:
         data = json.load(f)
    return data

In [55]:
def global_properties(G):
    n_nodes=nx.number_of_nodes(G)
    n_edges=nx.number_of_edges(G)
    apl=nx.average_shortest_path_length(G)
    adc=nx.average_degree_connectivity(G)
    acc=nx.average_clustering(G)
    d=nx.diameter(G)
    r=nx.radius(G)
    c=freeman_centralization(G)
    res = {"n_nodes": n_nodes, "n_edges": n_edges, 
           "average path length": apl, "average degree connectivity": adc, "average clustering connectivity": acc, 
          "diameter": d, "radius": r, "centralization": c}
    return res

def local_properties(G, flag=False):
    dc= nx.degree_centrality(G)
    bc= nx.betweenness_centrality(G)
    cc= nx.closeness_centrality(G)
    ec= nx.eigenvector_centrality_numpy(G)
    ratio= {k: bc[k]/dc[k] for k in dc}
    
    res= {"degree centrality": dc, "betweenness centrality": bc, "closeness centrality": cc, "eigenvalues centrality": ec, 
          "ratio": ratio}
    return res

def compute_graph(seed, flag="s"):
    if flag == "s":
        G = nx.from_pandas_dataframe(seed, 'SYMBOL_A', 'SYMBOL_B')
    else:
        G = nx.from_pandas_dataframe(seed, 'UNIPROT_AC_A', 'UNIPROT_AC_B')
    return G

def compute_LCC(G):
    LCC_G = max(nx.connected_component_subgraphs(G), key=len)
    return LCC_G

In [129]:
SGI = pd.read_csv("./interactions/interactome.csv")
I = pd.read_csv("./interactions/intersection_interactome.csv")
U = pd.read_csv("./interactions/union_interactome.csv")

In [131]:
G_SGI = compute_graph(SGI)
G_I = compute_graph(I)
G_U = compute_graph(U)

In [132]:
LCC_SGI = compute_LCC(G_SGI)
LCC_I = compute_LCC(G_I)
LCC_U = compute_LCC(G_U)

In [133]:
mapping_genes_U={i:j for i,j in enumerate(LCC_U.nodes())}
mapping_genes_I={i:j for i,j in enumerate(LCC_I.nodes())}
mapping_genes_inverse_U={j:i for i,j in enumerate(LCC_U.nodes())}

In [134]:
mapping = {"mapping_genes_U": mapping_genes_U, "mapping_genes_I": mapping_genes_I, "mapping_genes_inverse_U": mapping_genes_inverse_U}

## 2 clustering methods for disease modules discovery


* # simulated annealing

In [139]:
def convert_to_igraph(G):
    g = igraph.Graph(directed=False)
    g.add_vertices(G.nodes())
    g.add_edges(G.edges())
#     G_copy = copy.deepcopy(G)
#     A = nx.adjacency_matrix(G_copy).todense()
#     G_copy = igraph.Graph.Adjacency((A > 0).tolist())
#     G_copy.es['weight'] = A[A.nonzero()]
#     G_copy.to_undirected()

    return g

In [140]:
def clustering_simulated_annealing(G):
    G_i = convert_to_igraph(G)
    sa=G_i.community_spinglass()
    return sa

In [141]:
sa_I=clustering_simulated_annealing(LCC_I)

In [142]:
sa_U=clustering_simulated_annealing(LCC_U)

In [143]:
print(sa_I)

Clustering with 54 elements and 6 clusters
[0] M3K8, MP2K4, ITAV, TSP1, MK01
[1] FBXL4, SKP1, IKBA, FBX5
[2] P85B, STAT3, MYOD1, HDAC5, CBL, TWST1, MEF2D, EP300
[3] PGFRA, JAK1, IRF1, KS6B1, STAT1, FANCE, STAT2, FOS
[4] XRCC3, FLNA, BUB1B, MCPH1, BRCA2, PCNA, KAT2B, FANCG, BRCC3, GRWD1, SMC3,
    ERCC5, BCCIP
[5] KLHL3, GRIA1, GRP78, MLP3C, HSP74, KPCI, ISG15, FUS, SQSTM, INSR, TRAF6,
    GBRAP, ULK2, MTOR, DVL2, UBC


In [144]:
def routine(cluster):
    cluster=list(cluster)
    for c in range(len(cluster)):
        cluster[c] = mapping_genes_inverse_U[mapping_genes_I[cluster[c]]]
    return cluster

In [145]:
sa_U

In [146]:
sa_IJ={k: routine(sa_I[k]) for k in range(len(sa_I))}

sa_UJ={k: sa_U[k] for k in range(len(sa_U))}

In [148]:
annealing={"sa_I": sa_IJ, "sa_U": sa_UJ}

* # markov clustering

In [149]:
def markov_clustering(G):
    matrix = nx.to_scipy_sparse_matrix(G)
    result = mc.run_mcl(matrix)
    clusters = mc.get_clusters(result)  
    return clusters

In [150]:
mc_I = markov_clustering(LCC_I)

/home/georgos/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [151]:
mc_U = markov_clustering(LCC_U)

/home/georgos/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [152]:
mc_IJ={k: routine(mc_I[k]) for k in range(len(mc_I))}

mc_UJ={k: mc_U[k] for k in range(len(mc_U))}

In [153]:
markov={"mc_I": mc_IJ, "mc_U": mc_UJ}

 * # louvain clustering

In [154]:
def clustering_louvain(G):
    G_i = convert_to_igraph(G)
    lc=G_i.community_multilevel()
    return lc

In [155]:
lc_I=clustering_louvain(LCC_I)
lc_U=clustering_louvain(LCC_U)

In [156]:
lc_IJ={k: routine(lc_I[k]) for k in range(len(lc_I))}

lc_UJ={k: lc_U[k] for k in range(len(lc_U))}

In [157]:
louvain={"lc_I": lc_IJ, "lc_U": lc_UJ}

## load data

In [308]:
annealing=load_json("./annealing.json")
markov=load_json("./markov.json")
louvain=load_json("./louvain.json")

In [184]:
mapping=load_json("./mapping.json")

In [186]:
# mapping_genes_I=mapping["mapping_genes_I"]
# mapping_genes_inverse_U=mapping["mapping_genes_inverse_U"]
mapping_genes_U=mapping["mapping_genes_U"]

In [187]:
mc_U = markov["mc_U"]
mc_I = markov["mc_I"]

In [188]:
sa_U = annealing["sa_U"]
sa_I = annealing["sa_I"]

In [189]:
lc_U = louvain["lc_U"]
lc_I = louvain["lc_I"]

In [90]:
from scipy.misc import comb

In [287]:
n=300
M=6000
N=43
k=1

In [288]:
comb(n, k) * comb(M - n, N - k) / comb(M, N)

0.24922118774281532

# hypergeometric test

In [213]:
def hyper(k, M, n, N):
    '''
    M pop size
    N seed gene in pop
    
    n sample size
    k seed gene in sample
    '''
#     scipy.special.comb
#     num1=scipy.special.binom(K, k)
#     num1=np.float128(num1)
#     num2=scipy.special.binom((N-K), (n-k))
#     num2=np.float128(num2)
#     den=scipy.special.binom(N, n)
#     den=np.float128(den)
#     num=np.dot(num1, num2)
    #prb=hypergeom.pmf(k, K, n, N)
    #k, M, n, N
    #pmf=hypergeom.pmf(k, M, n, N)
    cdf=hypergeom.cdf(k, M, n, N)
    return 1-cdf

In [214]:
seed_index=set([int(i) for i in mapping_seed.keys()])

In [311]:
def hyp_test(mc, p_value):
    '''compute value hypergeometric at 95 % confidence
    if value inside this confidence accept hypothesis'''
    
    d={}
    M = sum([len(mc[i]) for i in mc])
    N = sum([len(set(mc[i]).intersection(seed_index)) for i in mc])
    for i in range(len(mc)):
        
        module_genes=mc[str(i)]
        seed_genes=list(set(mc[str(i)]).intersection(seed_index))
        n = len(module_genes)
        k = len(seed_genes)
        p_value=hyper(k, M, n, N)
        d[i] = [seed_genes, module_genes, p_value]
    return d

# putative disease modules

In [355]:
def routine_pdm(d, p_value):
    pdm={}
    for i in d:
        if d[i][2] < p_value:
            pdm[i] = d[i]
    return pdm

def putative_disease_modules(p_value=0.05):
    PDMs = {}
    for cluster in [markov, annealing, louvain]:
        for c in cluster:
            k=c.split("_")[1]
            cl=cluster[c]
            d=hyp_test(cl, p_value)
            pdm=routine_pdm(d, p_value)

            PDMs[c] = pdm
    return PDMs

In [356]:
PDMs=putative_disease_modules(p_value=0.05)

In [357]:
PDMs.keys()

dict_keys(['mc_U', 'sa_U', 'lc_I', 'mc_I', 'lc_U', 'sa_I'])

In [358]:
def routine_table(PDMs, flag="I"):
    table= {"clustering_algo":[], "index_module":[], "n_seed_genes":[], "n_genes":[], "ratio_s/g":[],  "p_value":[]}
    
    if flag == "I":
        lista=["lc_I", "mc_I", "sa_I"]
    else:
        lista=["lc_U", "mc_U", "sa_U"]
        
    for i in lista:
        temp=PDMs[i]
        keys=sorted(temp.keys())
        for k in keys:
            n_seed=len(temp[k][0])
            n_genes=len(temp[k][1])
            p_value=temp[k][2]
            
            if n_seed > 0:
                j=i.split("_")[0]
                if j == "lc":
                    table["clustering_algo"].append("louvain")
                elif j == "mc":
                    table["clustering_algo"].append("markov")
                else:
                    table["clustering_algo"].append("annealing")
                table["index_module"].append(k)
                table["n_seed_genes"].append(n_seed)
                table["n_genes"].append(n_genes)
                table["ratio_s/g"].append(n_seed/n_genes)
                table["p_value"].append(p_value)
    return table
        

In [368]:
table_module_I = routine_table(PDMs)
table_module_U = routine_table(PDMs, flag="U")

In [372]:
table_module_I=pd.DataFrame(table_module_I, columns=["clustering_algo", "index_module", "n_seed_genes", "n_genes", "ratio_s/g", "p_value"])
table_module_U=pd.DataFrame(table_module_U, columns=["clustering_algo", "index_module", "n_seed_genes", "n_genes", "ratio_s/g", "p_value"])

In [378]:
table_module_I.head()

,clustering_algo,index_module,n_seed_genes,n_genes,ratio_s/g,p_value
0,louvain,1,2,8,0.250000,0.035820
1,markov,6,1,3,0.333333,0.028778
2,markov,8,1,2,0.500000,0.010101
3,annealing,2,2,8,0.250000,0.035820


In [375]:
table_module_U.head()

,clustering_algo,index_module,n_seed_genes,n_genes,ratio_s/g,p_value
0,louvain,13,8,668,0.011976,0.035935
1,markov,0,1,36,0.027778,0.024973
2,markov,9,1,39,0.025641,0.028994
3,markov,13,1,27,0.037037,0.014469
4,markov,37,1,42,0.023810,0.033255


# 3 enrichment

In [379]:
import json
import pandas as pd
from bioservices import KEGG
from bioservices import UniProt

In [380]:
PDMs=load_json("./putative_disease_modules/PDMs.json")
mapping = load_json('./clustering/mapping.json')

In [381]:
import json
import requests

def retrieveUserListId(sym_genes):
    ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/addList'
    genes_str = '\n'.join(sym_genes)
    description = 'Example gene list'
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }
    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        raise Exception('Error analyzing gene list')
        
    data = json.loads(response.text)
    return data

In [382]:
def retrieveGO(data):
    ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/enrich'
    query_string = '?userListId=%s&backgroundType=%s'
    user_list_id = data['userListId']
    db = 'GO_Biological_Process_2017b'
    response = requests.get(
        ENRICHR_URL + query_string % (user_list_id, db)
     )
    if not response.ok:
        raise Exception('Error fetching enrichment results')

    GO = json.loads(response.text)
    return GO

In [383]:
def retrievePathway(data):
    ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/enrich'
    query_string = '?userListId=%s&backgroundType=%s'
    user_list_id = data['userListId']
    db = 'KEGG_2016'
    response = requests.get(
        ENRICHR_URL + query_string % (user_list_id, db)
     )
    if not response.ok:
        raise Exception('Error fetching enrichment results')

    path = json.loads(response.text)
    return path

In [384]:
def routineGO(data, module_index=[], clustering_method=[]):
    GO=retrieveGO(data)
    GO=GO["GO_Biological_Process_2017b"][:10]
    GO=[go[1:5] for go in GO]
    GO_df=pd.DataFrame(GO)
    GO_df.columns = ['GO category', 'P-value', 'Z-score', 'Combined score']
    if module_index != [] and clustering_method != []:
        GO_df["module index"] = module_index
        GO_df["clustering method"] = clustering_method
    return GO_df

In [393]:
def routinePathway(data, module_index=[], clustering_method=[]):
    pathway=retrievePathway(data)
    pathway=pathway["KEGG_2016"][:10]
    pathway=[path[1:6] for path in pathway]
    pathway_df=pd.DataFrame(pathway)
    pathway_df.columns = ['name', 'P-value', 'Z-score', 'Combined score', "overlapping genes"]
    if module_index != [] and clustering_method != []:
        pathway_df["module index"] = module_index
        pathway_df["clustering method"] = clustering_method
    return pathway_df

In [387]:
mapU=mapping["mapping_genes_U"]

In [388]:
clusters_U=["sa_U", "mc_U", "lc_U"]
clusters_I=["sa_I", "mc_I", "lc_I"]

In [389]:
methods=["simulated annealing", "markov clusters", "louvain clusters"]

In [390]:
def createTableGO(clusters):
    GO_list=[]
    i=0
    for cu in clusters:
        for module_index in PDMs[cu]:
            if len(PDMs[cu][module_index][1]) > 20:
                sym_genes=[mapU[str(s)] for s in PDMs[cu][module_index][1]]
                data=retrieveUserListId(sym_genes)
                go_df=routineGO(data, module_index, methods[i])
                GO_list.append(go_df)
        i+=1
    
    if GO_list == []:
        return None
    
    GO_df=GO_list[0]
    for go in GO_list[1:]:
        GO_df =pd.concat([GO_df, go])
    return GO_df

In [391]:
def createTablePathway(clusters):
    PATH_list=[]
    i=0
    for cu in clusters:
        for module_index in PDMs[cu]:
            if len(PDMs[cu][module_index][1]) > 20:
                sym_genes=[mapU[str(s)] for s in PDMs[cu][module_index][1]]
                data=retrieveUserListId(sym_genes)
                path_df=routinePathway(data, module_index, methods[i])
                PATH_list.append(path_df)
        i+=1
        
    if PATH_list == []:
        return None
    
    PATH_df=PATH_list[0]
    for path in PATH_list[1:]:
        PATH_df =pd.concat([PATH_df, path])
    return PATH_df

In [394]:
GO_df_U=createTableGO(clusters_U)
PATH_df_U=createTablePathway(clusters_U)

## alternative method to retrieve GO categories

In [102]:
def convert_uniprotID(seed):
    genes=[]
    for t in range(len(seed)):
        genes.append(mapping_genes_U[str(seed[t])]+"_HUMAN")
    return genes

In [103]:
def convert_kegg(genes):
    kegg_identifiers=u.mapping(fr="ID", to="KEGG_ID", query=genes)
    kegg_id = {}
    for k in kegg_identifiers:
        kegg_id[k] = kegg_identifiers[k][0]
    return kegg_id

In [104]:
def convert_uniprotAC(genes):
    uni_identifiers=u.mapping(fr="ID", to="ACC", query=genes)
    uni_id = {}
    for k in uni_identifiers:
        uni_id[k] = uni_identifiers[k][0]
    return uni_id

In [105]:
def routineGO(tt):
    uniprot_id=convert_uniprotID(tt)
    uni_ac = convert_uniprotAC(uniprot_id)
    uni_string=",".join(list(uni_ac.values()))
    res=s.Annotation(geneProductId=uni_string, taxonId=9606)
    return res

In [ ]:
s = QuickGO()

In [108]:
def compute_enrichment(PDMs, limit=10):
    enrichment={k:{} for k in PDMs}

    for clu in PDMs:
        cluster=PDMs[clu]
        for c in cluster:
            tt=PDMs[clu][c][1]
            res=routineGO(tt)

            enrichment[clu][c] = res["results"][:limit]
    return enrichment
enrichment=compute_enrichment(PDMs)

# netcarto

In [407]:
PMDs = load_json("./putative_disease_modules/PDMs.json")

In [415]:
def partitions(PDMs, LCC_I, LCC_U):
    
    lcc_i = pd.DataFrame(np.array(LCC_I.edges()))
    lcc_i.to_csv("lcc_i.dat",index=False, header=False, sep=" ")
    
    partition=set([mapping_genes_U[str(i)] for i in PDMs["sa_U"]["16"][1]])
    graph_partition=[]
    graph=LCC_U.edges()
    for g in graph:
        if g[0] in partition and g[1] in partition:
            graph_partition.append(g)
    partition=pd.DataFrame(np.array(graph_partition))
    
    partition.to_csv("lcc_u.dat",index=False, header=False, sep=" ")

In [416]:
partitions(PDMs, LCC_I, LCC_U)

# DIAMOnD

In [ ]:
res=pd.read_csv("./DIAMOnD/other/9606_noISI_Q3.txt", sep="\t")
apid=np.array(list(zip(res["GeneName_A"], res["GeneName_B"])) )
apid_df= pd.DataFrame(apid)
apid_df.to_csv("apid_df.csv", sep=" ", header=False, index=False)

In [ ]:
import zipfile
with zipfile.ZipFile("./DIAMOnD/BIOGRID-ORGANISM-3.4.155.mitab.zip", "r") as f:
    for name in f.namelist():
        if name == "BIOGRID-ORGANISM-Homo_sapiens-3.4.155.mitab.txt":
            data = f.read(name)
            print (name)

In [ ]:
name = "./DIAMOnD/BIOGRID-ORGANISM-Homo_sapiens-3.4.155.mitab.txt"
with open(name, "r") as f:
    data=f.read()

In [ ]:
data=data.split("\n")
data=[data[i].split("\t") for i in range(len(data))]

new_data=[[] for i in range(len(data[0]))]
for row in data[1:]:
    for i in range(len(row)):
        new_data[i].append(row[i])
        
data_dict={data[0][i]:new_data[i] for i in range(len(data[0]))}
data_dict["#ID Interactor A"] = data_dict["#ID Interactor A"][:-1]

In [ ]:
data_df=pd.DataFrame(data_dict)
data_df.to_csv("./DIAMOnD/bioGrid_df.csv")

In [ ]:
A=data_df["#ID Interactor A"]
B=data_df["ID Interactor B"]

A_biogrid=[]
for a in A.values:
    temp=a.split(":")[1]
    A_biogrid.append(temp)
    
B_biogrid=[]
for a in B.values:
    temp=a.split(":")[1]
    B_biogrid.append(temp)
    
biogrid_diamond=list(zip(A_biogrid, B_biogrid))
biogrid_diamond=np.array(biogrid_diamond)
biogrid_diamond=pd.DataFrame(biogrid_diamond)
biogrid_diamond.to_csv("biogrid_diamond.csv", header=False, sep=" ", index=False)

In [418]:
seed=load_json("../HW1/data/seed.json")

In [420]:
seed=seed["seed_AC"]

In [430]:
res_list=[]
u = UniProt(verbose=False)
for q in seed:
    res=u.mapping(fr="ACC", to="P_ENTREZGENEID", query=q)
    res_list.append(res)


# enrichment DIAMOnD

In [471]:
import bioservices as bi
u = bi.UniProt(verbose = False)

apid=pd.read_csv("./DIAMOnD/output/apid_200.txt", sep="\t")["DIAMOnD_node"].values
biogrid=pd.read_csv("./DIAMOnD/output/biogrid_200.txt", sep="\t")["DIAMOnD_node"].values

In [503]:
mapping_biogrid=u.mapping(fr="P_ENTREZGENEID", to="ID", query=biogrid)

In [519]:
biogrid_symbol=[mapping_biogrid[k][0].split("_")[0] for k in mapping_biogrid]

diamond_df = {"biogrid": biogrid_symbol[:40], "apid": apid[:40]}

diamond_df = pd.DataFrame(diamond_df)

In [523]:
diamond_df.head()

,apid,biogrid
0,PTPN2,B5B2P4
1,PML,RFA1
2,MAPK1,TAT
3,BRCA1,A0A0U3FYV6
4,STAT5A,A8K503


In [589]:
intersection=[]

biogrid_set=set(biogrid_symbol)
apid_set=set(apid)

for i in range(len(biogrid_symbol)):
    if biogrid_symbol[i] in apid_set:
        if biogrid_symbol[i] not in set(intersection):
            intersection.append(biogrid_symbol[i])
    if apid[i] in biogrid_set:
        if apid[i] not in set(intersection):
            intersection.append(apid[i])

In [593]:
hgnc = bi.HGNC()

In [595]:
gene_names=[]

In [596]:
for i in intersection:
    try:
        dictio = hgnc.fetch('symbol', i)
        gene_name = dictio['response']['docs'][0]["name"]

        gene_names.append((i, gene_name))
    except:
        print(i)
        continue

### GO and Pathways 

In [604]:
intersection_seed=list(set(intersection).union(set(seed)))

In [606]:
data=retrieveUserListId(intersection_seed)
go_DIAMOnD=routineGO(data)
pathway_DIAMOnD=routinePathway(data)